# Probabilistic Programming 3: Hidden Markov modeling
## Variational inference

In this notebook, we are looking at inference in a dynamical system.

### Preliminaries

- Goal 
  - Learn to apply ForneyLab to a dynamical system.
- Materials        
  - Mandatory
    - These lecture notes.
  - Optional
    - Cheatsheets: [how does Julia differ from Matlab / Python](https://docs.julialang.org/en/v1/manual/noteworthy-differences/index.html).
    - Getting started with [ForneyLab](https://biaslab.github.io/forneylab/docs/getting-started/).

In [3]:
using Pkg;Pkg.activate("workspace");Pkg.instantiate()

Activating environment at `~/Documents/biaslab/repos/BMLIP/lessons/notebooks/probprog/workspace/Project.toml`


In [ ]:
using Random
using Distributions
using Plots
pyplot()
include("../scripts/pp-3.jl") 

Random.seed!(1234);

### Generate data

In [ ]:
# There are 3 possible states and each variable is in one of those (one-hot encoding)
K = 3

# Length of time-series
T = 50

# Transition matrix of latent variables
transition = [0.3 0.6 0.1; 
              0.5 0.2 0.3; 
              0.2 0.8 0.1]

# Emission matrix for observed variables
emission = [0.7 0.3 0.0; 
            0.2 0.6 0.2; 
            0.0 0.3 0.7]

# Preallocate data arrays
X = zeros(T+1, K)
Y = zeros(T, K)

# Initial state
X[1,:] = [0.0, 1.0, 0.0] 

# Generate data for entire time-series
for t = 2:T
    
    # Transition from previous state
    A = transition * X[t-1,:]
    
    # Sample from Categorical distribution
    X[t,:] = one_hot(rand(Categorical(A ./ sum(A)), 1)[1], K)
    
    # Emission of current state
    B = emission * X[t,:]
    
    # Sample from Categorical distribution
    Y[t-1,:] = one_hot(rand(Categorical(B ./ sum(B)), 1)[1], K)
    
end

# For visualization, we collapse the data from a one-hot to a numerical encoding
states = argmax.(eachrow(X))
observations = argmax.(eachrow(Y))

# Visualization.
plot(1:T, states[2:end], color="red", label="states", ylim=(0, 4), grid=false)
scatter!(1:T, observations, color="blue", label="observations")
xlabel!("time (t)")

### Model specification

Let's quickly recap the $K$-component Gaussian mixture model. 

We have a data set of $X$ of $N$ samples by $D$ features. Each component is a Gaussian distribution, $ p_k(x \mid \mu_k, \Sigma_k) \triangleq \mathcal{N}(x \mid \mu_k, \Sigma_k)$. In total, we model $N$ samples:

$$ p(X \mid z, \mu, \Sigma) = \sum_{i=1}^{N} \sum_{k=1}^{K} p_k(x_i \mid \mu_k, \Sigma_k)^{z_i=k} \, .$$

<!-- Note that we summarize parameters as follows: $\phi = (\phi_1, \dots, \phi_K), \mu = (\mu_1, \dots, \mu_K), \Sigma = (\Sigma_1, \dots, \Sigma_K)$.  -->
The variable $z_i$ is the assignment of sample $i$ to class $k$. The probability $p(z_{i} = k)$ is also called the _responsibility_ of component $k$ for the $i$-th sample. All $z_i$ follow a Categorical distribution with mixture weights as parameters:

$$ p(z_i) \sim \text{Cat}(\phi) \, ,$$

Being Bayesians, we put priors on all unknown parameters ($\phi, \mu, \Sigma$):

$$ \begin{align}
p(\phi) \sim&\ \text{Dir}(a_0) \\
p(\mu) \sim&\ \mathcal{N}(m_0, l_0^{-1}\Sigma) \\
p(\Sigma) \sim&\ \mathcal{W}(W_0, v_0) \end{align} \, .$$

The distribution for the mixture weights, $p(\phi)$, follows a Dirichlet distribution because it is a vector that should sum to one. The component parameter priors are a Wishart distribution for the covariance matrix and a Gaussian distribution for the mean, conditioned on the sampled covariance matrix. Together, these two priors form a [Normal-Inverse-Wishart distribution](https://en.wikipedia.org/wiki/Normal-inverse-Wishart_distribution), $p(\mu, \Sigma)$, which is the conjugate prior for the Gaussian components. The full joint distribution is therefore:

$$ \begin{align} p(X, z, \phi, \mu, \Sigma) =&\ p(X \mid z, \phi, \mu, \Sigma) p(z \mid \phi) p(\mu, \Sigma) p(\phi) \\
=&\ \sum_{i=1}^{N} \sum_{k=1}^{K} \mathcal{N}(x_i \mid \mu_k, \Sigma_k)^{z_i=k}\ \text{Cat}(z_i=k \mid \phi_k)\ \mathcal{NIW}(\mu_k, \Sigma_k \mid m_{0k}, l_{0k}, W_{0k}, v_{0k})\ \text{Dir}(\phi_k \mid a_{0k}) \end{align}$$

We are interested in the posterior distribution over $z$, which assigns the current samples to clusters via $\arg \max_{k} p(z_i = k)$. For example, if the posterior distribution for $z_i$ is $[0.32\ 0.21\ 0.47]$, then $\hat{k}_i$ is 3 and we say that $x_i$ belongs to the component 3. We are also interested in the posterior distributions for the component parameters, because these allow us to assign future samples to the current clusters. The posterior for $z, \phi, \mu, \Sigma$ has the form:

$$p(z, \phi, \mu, \Sigma \mid X) = \frac{p(z, \phi, \mu, \Sigma \mid X)}{\int p(X, z, \phi, \mu, \Sigma)\ \mathrm{d}X} \, .$$

We cannot evaluate the integral in the denominator, but because it only serves as a normalization factor we have a workaround. Note that the numerator is the full joint distribution and that if you fix the parameters $\phi, \mu, \Sigma$ to certain values, it simplifies to:

$$p(z_i \mid X) = \sum_{i=1}^{N} \sum_{k=1}^{K} \rho_k^{z_i = k} \, ,$$

where $\rho_k = \phi_k \mathcal{N}(x_i \mid \mu_k, \Sigma_k)$ for all $k$. This is a Categorical distribution! The only problem is that $\rho$ is not normalized correctly. But we know that it _should_ be correctly normalized, after division by the denominator term above, $\int p(X, z, \phi, \mu, \Sigma) \ \mathrm{d}X$. We can therefore perform the following re-parameterization:

$$r_k = \frac{\rho_k}{\sum_{k} \rho_k} \, .$$

Now, $r_k$ are valid responsibilities for the assignment variable $z$. In other words, $p(z_i \mid X) \sim \text{Cat}(r_k)$. But how should we fix $\phi, \mu, \Sigma$? <br>
Note that if we fix $z_i$ to $k=2$ for instance, the full posterior simplifies to:

$$p(\phi_2, \mu_2, \Sigma_2 \mid X) = \sum_{i=1}^{N} \mathcal{N}(x_i \mid \mu_2, \Sigma_2) \ \text{Cat}(z_i=2 \mid \phi_2)\ \mathcal{NIW}(\mu_2, \Sigma_2 \mid m_{02}, l_{02}, W_{02}, v_{02}) \ \text{Dir}(\phi_2 \mid a_{02}) \, .$$

This can be split according to $p(\phi) p(\mu, \Sigma)$:

$$\begin{align} 
p(\phi_k \mid z) =&\ \sum_{i=1}^{N} \text{Cat}(z_i \mid \phi_k)\ \text{Dir}(\phi_k \mid a_{0}) \\ 
p(\mu_k, \Sigma_k \mid X) =&\ \sum_{i=1}^{N} \mathcal{N}(x_i \mid \mu_k, \Sigma_k)\ \mathcal{NIW}(\mu_k, \Sigma_k \mid m_{0k}, l_{0k}, W_{0k}, v_{0k}) 
\end{align}$$

which are Dirichlet and a Normal-Inverse-Wishart distributions. So, we have multiple marginal posteriors with known forms. We can define a recognition distribution that takes a similar form.

In [ ]:
using ForneyLab

In [ ]:
g = FactorGraph()

@RV A ~ Dirichlet(ones(3,3)) # Vague prior on transition model
@RV B ~ Dirichlet([10.0 1.0 1.0; 1.0 10.0 1.0; 1.0 1.0 10.0]) # Stronger prior on observation model
@RV s_0 ~ Categorical(1/3*ones(3))

s = Vector{Variable}(undef, n_samples) # one-hot coding
x = Vector{Variable}(undef, n_samples) # one-hot coding
s_t_min = s_0
for t = 1:n_samples
    @RV s[t] ~ Transition(s_t_min, A)
    @RV x[t] ~ Transition(s[t], B)
    
    s_t_min = s[t]
    
    placeholder(x[t], :x, index=t, dims=(3,))
end;

### Algorithm Generation

In [ ]:
# Define the recognition factorization
q = RecognitionFactorization(A, B, [s_0; s], ids=[:A, :B, :S])

# Generate VMP algorithm
algo = variationalAlgorithm(q)

# Construct variational free energy evaluation code
algo_F = freeEnergyAlgorithm(q);

### Execution

In [ ]:
# Load algorithms
eval(Meta.parse(algo))
eval(Meta.parse(algo_F))

# Initial recognition distributions
marginals = Dict{Symbol, ProbabilityDistribution}(
    :A => vague(Dirichlet, (3,3)),
    :B => vague(Dirichlet, (3,3)))

# Initialize data
data = Dict(:x => x_data)
n_its = 20

# Run algorithm
F = Vector{Float64}(undef, n_its)
for i = 1:n_its
    stepS!(data, marginals)
    stepB!(data, marginals)
    stepA!(data, marginals)

    F[i] = freeEnergy(data, marginals)
end
;

In [ ]:
### Plot results

In [ ]:
using PyPlot

# Plot free energy
plot(1:n_its, F, color="black")

grid("on")
xlabel("Iteration")
ylabel("Free Energy")
xlim(0,n_its);

In [ ]:
figure(figsize=(10,5))

# Collect state estimates
x_obs = [findfirst(x_i.==1.0) for x_i in x_data]
s_true = [findfirst(s_i.==1.0) for s_i in s_data]

# Plot simulated state trajectory and observations
subplot(121)
plot(1:n_samples, x_obs, "k*", label="Observations x", markersize=7)
plot(1:n_samples, s_true, "k--", label="True state s")
yticks([1.0, 2.0, 3.0], ["Red", "Green", "Blue"])
grid("on")
xlabel("Time")
legend(loc="upper left")
xlim(0,n_samples)
ylim(0.9,3.1)
title("Data set and true state trajectory")

# Plot inferred state sequence
subplot(122)
m_s = [mean(marginals[:s_*t]) for t=1:n_samples]
m_s_1 = [m_s_t[1] for m_s_t in m_s]
m_s_2 = [m_s_t[2] for m_s_t in m_s]
m_s_3 = [m_s_t[3] for m_s_t in m_s]

fill_between(1:n_samples, zeros(n_samples), m_s_1, color="red")
fill_between(1:n_samples, m_s_1, m_s_1 + m_s_2, color="green")
fill_between(1:n_samples, m_s_1 + m_s_2, ones(n_samples), color="blue")
xlabel("Time")
ylabel("State belief")
grid("on")
title("Inferred state trajectory");

In [ ]:
# True state transition probabilities
PyPlot.plt.matshow(A_data, cmap="bone", vmin=0.0, vmax=1.0)
ttl = title("True state transition probabilities")
ttl.set_position([.5, 1.15])
yticks([0, 1, 2], ["Red", "Green", "Blue"])
xticks([0, 1, 2], ["Red", "Green", "Blue"], rotation="vertical")
colorbar()

# Inferred state transition probabilities
PyPlot.plt.matshow(mean(marginals[:A]), cmap="bone", vmin=0.0, vmax=1.0)
ttl = title("Inferred state transition probabilities")
ttl.set_position([.5, 1.15])
yticks([0, 1, 2], ["Red", "Green", "Blue"])
xticks([0, 1, 2], ["Red", "Green", "Blue"], rotation="vertical")
colorbar();